In [9]:
import numpy as np
import pickle as pk
import pandas as pd
import copy
import MinimaxTestAB
from IPython.display import display
# Note that pickle doesn't actually store information about how a class/object is constructed, 
# and needs access to the class when unpickling.

# A snapshot of a single frame within a game along with the final win_outcome at that frame
class BoardState:
    def __init__(self, board_state, win_outcome_list = []):
        self.board_state = board_state
        self.win_outcome_list = win_outcome_list
        
    def get_board_state(self):
        return self.board_state
    
    def get_win_outcome_list(self):
        return self.win_outcome_list
    
    def add_win_outcome_to_list(self, new_win_outcome):
        self.win_outcome_list.append(new_win_outcome)

# Contains the entire game in a list of BoardState objects
class OthelloGameStates:
    board_state_obj_list = []
    
    def __init__(self, board_state_obj):
        self.board_state_obj_list.append(board_state_obj)
        
    def add_board_state_obj_to_list(self, board_state_obj):
        self.board_state_obj_list.append(board_state_obj)
        
    def get_board_state_obj_list(self):
        return self.board_state_obj_list
    
    def get_count(self):
        return len(self.board_state_obj_list)

In [10]:
# Reducer reads in game states and keeps those that have more than 1 instance in the win_outcome_list
# indicating a duplicate that can be used to find the win likelihood

# Reduced dataset is written to "othello_game_states_reduced.pickle"
new_game_states_dict = {}
try:
    with open("othello_game_states.pickle","rb") as pickle_in:
        with open("othello_game_states_reduced.pickle","wb") as pickle_out:
            db_game_states_dict = pk.load(pickle_in)
            for key in db_game_states_dict.keys():
                state = db_game_states_dict[key]
                # If duplicate states OR if state is an endgame
                if (len(state.get_win_outcome_list()) > 1 
                    or len(MinimaxTestAB.successors(state.get_board_state(), 0)) == 0 
                    or len(MinimaxTestAB.successors(state.get_board_state(), 1)) == 0):
                    new_game_states_dict[key] = copy.deepcopy(db_game_states_dict[key])
                    
            pk.dump(new_game_states_dict, pickle_out)
except IOError:
    print("IOError", file=sys.stderr)
except EOFError:
    print("EOFError", file=sys.stderr)

In [11]:
# Calculates win likelihood based on total number of games and porportion of wins for black
# Creates new input file to ValueNet with:
# 1. Grid divided into 3 feature planes as x-input
# 2. Padded borders so ValueNet can recognize corners and edges
# 3. Win-likelihood as target label (y-input)

In [12]:
value_net_x_input_list = []
value_net_y_input_list = []
for key in new_game_states_dict.keys():
    state = db_game_states_dict[key]
    total = len(state.get_win_outcome_list())
    black_count = 0.0
    win_likelihood = -1.0
    for item in state.get_win_outcome_list():
        if item == 0:
            black_count += 1.0
    win_likelihood = black_count/total
    
    value_net_x_input_list_features = []
    value_net_x_input_list_empty_feature = [[0 for col in range(10)] for row in range(10)]
    value_net_x_input_list_black_feature = [[0 for col in range(10)] for row in range(10)]
    value_net_x_input_list_white_feature = [[0 for col in range(10)] for row in range(10)]
    
    # Divide 1 feature plane into 3 by going over every cell, 
    # get id and then add cell to appropriate feature plane (which is already enlarged)
    for i in range(8):
        for j in range(8):
            if state.get_board_state()[i][j] == -1:
                value_net_x_input_list_empty_feature[i+1][j+1] = 1
            elif state.get_board_state()[i][j] == 0:
                value_net_x_input_list_black_feature[i+1][j+1] = 1
            elif state.get_board_state()[i][j] == 1:
                value_net_x_input_list_white_feature[i+1][j+1] = 1
                
    # Individual feature planes are constructed. Append to list_features list
    value_net_x_input_list_features.append(value_net_x_input_list_empty_feature)
    value_net_x_input_list_features.append(value_net_x_input_list_black_feature)
    value_net_x_input_list_features.append(value_net_x_input_list_white_feature)
    
    # Add depth-3 list_features to input_list to represent a single frame
    value_net_x_input_list.append(value_net_x_input_list_features)
    value_net_y_input_list.append(win_likelihood)

with open("value_net_x_input.pickle","wb") as pickle_out:
    pk.dump(value_net_x_input_list, pickle_out)
    
with open("value_net_y_input.pickle","wb") as pickle_out:
    pk.dump(value_net_y_input_list, pickle_out)

In [14]:
np.array(value_net_y_input_list).shape

(302,)